In [ ]:
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

sys.path.append(os.path.abspath(os.path.join('../src')))
from varying_drift_diffusion import *
from motion_simulation import *

# bayesflow
sys.path.append(os.path.abspath(os.path.join('../../BayesFlow')))
# sys.path.append(os.path.abspath(os.path.join('../../../BayesFlow')))
from bayesflow.networks import InvariantNetwork, InvertibleNetwork
from bayesflow.amortizers import SingleModelAmortizer
from bayesflow.trainers import ParameterEstimationTrainer
from bayesflow.diagnostics import *

from tensorflow.keras.layers import Dense, GRU, LSTM, Conv1D, MultiHeadAttention, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential

In [ ]:
%load_ext autoreload
%autoreload 2
np.set_printoptions(suppress=True)

## Constants

In [ ]:
# simulation
N_SIM = 1000
N_OBS = 100

# bayesflow
PARAM_NAMES = ["a", "ndt", "bias", "kappa"]
N_PARAMS = len(PARAM_NAMES)
N_EPOCHS = 30
ITER_PER_EPOCH = 1000
BATCH_SIZE = 32
N_SAMPLES = 2000

## Simulator Test

In [ ]:
n_obs = 100

a     = 1.0
ndt   = 0.2
bias  = 0.5
kappa = 0.1
theta = np.array([a, ndt, bias, kappa])

motion_profiles, condition = motion_experiment(n_obs, n_unique_motions=5, motion_dur=2)
rt, resp = var_dm_simulator(theta, n_obs, motion_profiles)


In [ ]:
%time
p_, x_ = var_dm_batch_simulator(32, 100)
x_.shape

## Bayes Flow

In [ ]:
# default summary network
summary_net = InvariantNetwork()

In [ ]:
# invertable inference network
meta_dict={
    'n_coupling_layers': 5,
    's_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    't_args': {
        'units': [128, 128],
        'activation': 'elu',
        'initializer': 'glorot_uniform',
    },
    'alpha': 1.9,
    'permute': True,
    'n_params': N_PARAMS
}

inference_net = InvertibleNetwork(meta_dict)

In [ ]:
# Connect summary and inference network
amortizer = SingleModelAmortizer(inference_net, summary_net)

# Learning-rate decay
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(
    0.0005, 1000, 0.99, staircase=True
)

trainer = ParameterEstimationTrainer(
    network=amortizer, 
    generative_model=var_dm_batch_simulator,
    learning_rate=learning_rate,
    checkpoint_path='checkpoints/time_var_dm',
    clip_value=3,
    max_to_keep=5
)

In [ ]:
%%time
# online training
losses = trainer.train_online(N_EPOCHS, ITER_PER_EPOCH, BATCH_SIZE, n_obs=N_OBS)

## Parameter Recovery